### [2] Caesarian 제왕절개 Logistic Regression 모델을 텐서플로를 사용하여 3층으로신경망을 구현하여 Accuracy를 구하여 단층 신경망과 비교하여 보세요
###    단층일 때보다 정확도가 높아지도록 구현한다

In [1]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(5)

In [2]:
# 데이터 불러 오기
xy = np.loadtxt('caesarian.csv', delimiter=',', dtype=np.float32)
xy.shape        #(759,9)
print(xy.shape)
# 학습 데이터 분리 : 70% 531개
x_train = xy[:56,:-1]     # X     
y_train = xy[:56,[-1]]    # Y
print(x_train.shape,y_train.shape)

# 검증 데이터 분리 : 30% 228개
x_test = xy[56:,:-1]     # X     
y_test = xy[56:,[-1]]    # Y
print(x_test.shape,y_test.shape)

(80, 6)
(56, 5) (56, 1)
(24, 5) (24, 1)


In [3]:
# 변수 초기화 : weight, bias
# (m,n) * (n,l) = (m,l)  : 행렬의 내적 곱셉 공식
#(56,5) * (5,5) = (56,5)                                 # x_train.shape과 W를 행렬곱하여 y_train.shape되도록
W1 = tf.Variable(tf.random.normal([5,3]),name='weight1')     # W의 행렬 수치 수정(random.normal([ 여기 값 ])) 
b1 = tf.Variable(tf.random.normal([3]),name='bias1')         # b(bias)는 W의 마지막값과 같은 값으로 둔다 

def layer1(X):
    return tf.nn.relu(tf.matmul(X,W1) + b1)

In [4]:
# layer2
# 변수 초기화 : weight, bias
# X * W = Y
# (m,n) * (n,l) = (m,l)  : 행렬의 내적 곱셉 공식
#(56,5) * (5,5) = (56,5)                                 # x_train.shape과 W를 행렬곱하여 y_train.shape되도록
W2 = tf.Variable(tf.random.normal([3,2]),name='weight2')     # W의 행렬 수치 수정(random.normal([ 여기 값 ])) 
b2 = tf.Variable(tf.random.normal([2]),name='bias2')         # b(bias)는 W의 마지막값과 같은 값으로 둔다 

def layer2(X):
    return tf.nn.relu(tf.matmul(layer1(X),W2) + b2)

In [5]:
# Layer 3 : 출력 층(Output layer)
W3 = tf.Variable(tf.random.normal([2,1]),name='weight3')     # W의 행렬 수치 수정(random.normal([ 여기 값 ])) 
b3 = tf.Variable(tf.random.normal([1]),name='bias3')         # b(bias)는 W의 마지막값과 같은 값으로 둔다 

In [6]:
# 예측 함수(hypothesis) : H(X) = sigmoid(W*X + b)
# 이전 층의 출력이 입력으로 들어와야됨 
def hypothesis(X):
    return tf.sigmoid(tf.matmul(layer2(X),W3) + b3)  # 0 과 1 사이의 값 출력

In [7]:
# 비용 함수 : logloss, 2진 분류 모델
def cost_func():
    cost = -tf.reduce_mean(y_train*tf.math.log(hypothesis(x_train)) +                # y가 0일때는 앞의식은 0됨 
                         (1-y_train)*tf.math.log(1-hypothesis(x_train)))             # y가 1일때는 뒤의식은 0됨 
    return cost

In [8]:
# 경사 하강법
# learning_rate(학습율)을 0.01 로 설정하여 optimizer객체를 생성
optimizer = tf.optimizers.Adam(learning_rate=0.01)

In [9]:
# 학습 시작
print('***** Start Learning!!')
for step in range(10001):
    optimizer.minimize(cost_func,var_list=[W1,b1,W2,b2,W3,b3])
    if step % 1000 == 0:
        print('%04d'%step,'cost:[',cost_func().numpy(),']')
        
print('***** Learning Finished!!')

***** Start Learning!!
0000 cost:[ 0.70376265 ]
1000 cost:[ 0.46454245 ]
2000 cost:[ 0.46268204 ]
3000 cost:[ 0.4625016 ]
4000 cost:[ 0.4632113 ]
5000 cost:[ 0.46248025 ]
6000 cost:[ 0.690594 ]
7000 cost:[ 0.690594 ]
8000 cost:[ 0.690594 ]
9000 cost:[ 0.690594 ]
10000 cost:[ 0.690594 ]
***** Learning Finished!!


In [10]:
# 예측
print('***** Predict')

tx_test = np.array(x_test,dtype=np.float32)
print(hypothesis(tx_test))

***** Predict
tf.Tensor(
[[0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]
 [0.5357143]], shape=(24, 1), dtype=float32)


In [11]:
#정확도 측정 : accuracy computation
def predict(X):
    return tf.cast(hypothesis(X) > 0.5,dtype=tf.float32)

# 검증 데이터를 사용하여 예측 
preds = predict(x_test)
accuracy = tf.reduce_mean(tf.cast(tf.equal(preds,y_test),dtype=tf.float32))

print('Accuracy:',accuracy.numpy()) 
# print('Hypothesis:\n',hypothesis(x_test).numpy())
# print('Predict:\n',preds.numpy())

Accuracy: 0.6666667
